# QCCS: This is a simple demonstration of finding the best "sample delay" value for your UHF-QA

## The script runs multiple measurements in steps of 4 from 0 to 1020
## In each measurement, the absolute value is read out from integrated I/Q value
### At the end of iteration, the maximum absolute value indicates the sample delay best suited; present in your system

### Table of Contents

* [1) Imports and helper functions](#chapter1)
* [2) Device initialization/connection](#chapter2)
* [3) Configure PQSC](#chapter3)
* [4) Configure PQSC](#chapter4)
* [5) Configure UHFQA](#chapter5)
    * [5.2 Configure UHFQA QCCS mode](#section_5_1)
    * [5.2 Configure UHFQA for 2-qubit](#section_5_2)
* [6) Configure HDAWG](#chapter6)
* [7) Configure Upload and Run program](#chapter7)
* [8) Read results (under work)](#chapter8)

### 1) Imports and helper functions <a class="anchor" id="chapter1"></a>

In [1]:
import time
import zhinst.ziPython as zi
import numpy as np
import matplotlib.pyplot as plt
import math

# Compile AWG seqC code
def awg_compile_upload_elf(awgModule, awgIndex, awg_program):
    """Compile and upload awg_program as .elf file"""
    awgModule.set('awgModule/index', awgIndex)
    awgModule.set('awgModule/compiler/sourcestring', awg_program)

    while awgModule.getInt('awgModule/compiler/status') == -1:
        time.sleep(0.1)
    if awgModule.getInt('awgModule/compiler/status') == 1:
        raise Exception(awgModule.getString('awgModule/compiler/statusstring'))

    if awgModule.getInt('awgModule/compiler/status') == 2:
        print("Compilation successful with warnings, will upload the program to the instrument.")
        print("Compiler warning: ", awgModule.getString('awgModule/compiler/statusstring'))

    time.sleep(0.2)
    i = 0
    while (awgModule.getDouble('awgModule/progress') < 1.0) and (awgModule.getInt('awgModule/elf/status') != 1):
        time.sleep(0.5)
        i += 1

    if awgModule.getInt('awgModule/elf/status') == 1:
        raise Exception("Upload to the instrument failed.")
        
# Rotation in I/Q plane
def rotate(number, degree):
    n = number
    return (math.e**(math.radians(degree)*1j)) * n

### 2) Specify your data server + devices used <a class="anchor" id="chapter2"></a>
#### Note: variable UHFQA_watch can be ignored

In [135]:
SERVER = '127.0.0.1'

daq = zi.ziDAQServer(SERVER, 8004, 1)

setupNames = ['Continuity', 'Frankenstein']
setupIDs = [['dev10006','dev2171',['dev8147']], ['dev10004','dev2033',['dev8033']]]

# JS: Choose 0 for Continuity, 1 for Frankenstein setup
runSetup = 1

print(f'Setup run used: {setupNames[runSetup]}')

PQSC = setupIDs[runSetup][0]
UHFQA = setupIDs[runSetup][1]
HDAWG = setupIDs[runSetup][2][0]

# On the continuity setup, this device is not connected to anything or perform any function
UHFQA_watch = 'dev2004' 


# Connection to data server
daq.connectDevice(PQSC, '1gbe')
daq.connectDevice(UHFQA, '1gbe')
daq.connectDevice(HDAWG, '1gbe')

Setup run used: Frankenstein


### 3) Configure the PQSC for register forwarding (conditional unit TBD)  <a class="anchor" id="chapter3"></a>

In [ ]:
### Configure PQSC

# Use external reference clock coming from the UHFQA
daq.setInt(f'/{PQSC}/system/clocks/referenceclock/in/source', 1)

## Configure execution engine
# Send a single trigger to start the HDAWG and the UHFQA
daq.setInt(f'/{PQSC}/execution/repetitions', 1)
# Wait for feedback to arrive from UHFQA (10us is more than enough)
daq.setDouble(f'/{PQSC}/execution/holdoff', 10e-6)

## Register forwarding
# ZSync output port to the receiver HDAWG
port = 0
# Program register bank forwarding
fwd_length = 4
fwd = range(0, 4)
daq.setVector(f'/{PQSC}/raw/regs/{port}/fwd', np.array(fwd).astype(np.uint32))
# Enable forwarding on output port
daq.setInt(f'/{PQSC}/raw/zsyncs/{port}/txmux/fwd_en', 1)

## Condition unit
# ZSync output port to the receiver HDAWG
port = 0
# Program condition unit sources
src_length = 8
src = range(0, 8)
daq.setVector(f'/{PQSC}/raw/cond/src', np.array(src).astype(np.uint32))
# Program condition unit function
lut_length = 2**src_length
offset = 512
lut = range(offset, offset+lut_length)
daq.setVector(f'/{PQSC}/raw/cond/lut', np.array(lut).astype(np.uint32))
# Program condition unit byte selection
sel_length = 18
sel = [1] * sel_length
daq.setVector(f'/{PQSC}/raw/cond/sel', np.array(sel).astype(np.uint32))

# Enable condition output on forwarding port
daq.setInt(f'/{PQSC}/raw/zsyncs/{port}/txmux/cond_en', 0) # This should be 0 according to Niels, cond not working

### 4) Configure HDAWG to QCCS mode  <a class="anchor" id="chapter4"></a>

In [ ]:
## Configure HDAWG

# Use ZSync clock
daq.setInt(f'/{HDAWG}/system/clocks/referenceclock/source', 2)

## Configure DIO
# Configure DIO switch to QCCS mode
daq.setInt(f'/{HDAWG}/dios/0/mode', 3)
# Drive the two most significant bytes of the DIO port
# The UHFQA drives the two least significant bytes of the DIO port
daq.setInt(f'/{HDAWG}/dios/0/drive', 0b1100)
# Configure DIO triggering to match ZSync input
daq.setInt(f'{HDAWG}/awgs/0/dio/strobe/slope', 0)
daq.setInt(f'{HDAWG}/awgs/0/dio/valid/polarity', 0)

## Configure AWG
# Setup AWG module
awg_hd = daq.awgModule()
awg_hd.set('device', HDAWG)
awg_hd.set('index', 0)
awg_hd.execute()

# Turn on HDAWG outputs
daq.setInt(f'/{HDAWG}/sigouts/0/on', 1)
daq.setInt(f'/{HDAWG}/sigouts/1/on', 1)

### 5) Configure UHFQA <a class="anchor" id="chapter5"></a>

### 5.1) Configure UHFQA for QCCS mode (underwork) <a class="anchor" id="section_5_1"></a>

In [ ]:
### Configure UHFQA

# UHFQA uses its own clock in this experiment

## Configure DIO
# Sample DIO data at 50 MHz
daq.setInt(f'{UHFQA}/dios/0/extclk', 2)
# Set DIO output to QA result QCCS
daq.setInt(f'{UHFQA}/dios/0/mode', 4)
# Drive the two least significant bytes of the DIO port
# The HDAWG drives the two most significant bytes of the DIO port
daq.setInt(f'{UHFQA}/dios/0/drive', 0b0011)
# Configure DIO triggering to match HDAWG DIO input
daq.setInt(f'/{UHFQA}/awgs/0/dio/strobe/slope', 0)
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/index', 16)

## QA readout configuration
# Bypass crosstalk to reduce latency
daq.setInt(f'{UHFQA}/qas/0/crosstalk/bypass', 1)
# Set length of integration (arbitrary value of 128)
daq.setInt(f'{UHFQA}/qas/0/integration/length', 128)
# Set rotations all to 1+0i
for i in range(10):
    daq.setComplex(f'/{UHFQA}/qas/0/rotations/0', 1)
# Reset QA results
daq.setInt(f'{UHFQA}/qas/0/result/reset', 1)

### 5.2) Configure UHFQA for two-qubit readout + upload integration weights <a class="anchor" id="section_5_2"></a>

In [ ]:
# Input/Output settings
daq.setInt(f'/{UHFQA}/sigins/0/imp50', 1)
daq.setInt(f'/{UHFQA}/sigins/1/imp50', 1)
daq.setDouble(f'/{UHFQA}/sigins/0/range', 1.5)
daq.setDouble(f'/{UHFQA}/sigins/1/range', 1.5)
daq.setInt(f'/{UHFQA}/sigouts/0/on', 1)
daq.setInt(f'/{UHFQA}/sigouts/1/on', 1)


# Configure QA setup
### NOTE: Hardcoded sample delay of 268 samples, depends on length of your wires and the delay present
#daq.setInt(f'/{UHFQA}/qas/0/delay', 272) # JS: Need to adjust this accordingly to whatever setup we use. 276 is what I found optimal on Continuity setup 7/13/2020. 268 is for my setup
daq.setDouble(f'/{UHFQA}/qas/0/integration/length', 128) # JS: Does reducing the length = less time used for integration? Probably not
daq.setInt(f'/{UHFQA}/qas/0/integration/mode', 0)
daq.setInt(f'/{UHFQA}/qas/0/integration/sources/0', 0)
daq.setInt(f'/{UHFQA}/qas/0/integration/sources/1', 0)

# Set QA result mode to "Integration" mode in I/Q plane
daq.setInt(f'/{UHFQA}/qas/0/result/source', 7)


# Upload integration weights
ch1_freq = 28.125e6 # Oscillator frequency
ch2_freq = 56.25e6 # Oscillator frequency
fs_uhfqa = 1.8e9 # Sampling frequency of UHF-QA
integrationPoints = 4096 # Number of sampled used for integration
ampl = 0.38 # Vpk of weights signal

ch1_w_real = ampl*np.sin(2*np.pi*ch1_freq*np.arange(integrationPoints)/fs_uhfqa)
ch1_w_imag = ampl*np.cos(2*np.pi*ch1_freq*np.arange(integrationPoints)/fs_uhfqa)

ch2_w_real = ampl*np.sin(2*np.pi*ch2_freq*np.arange(integrationPoints)/fs_uhfqa)
ch2_w_imag = ampl*np.cos(2*np.pi*ch2_freq*np.arange(integrationPoints)/fs_uhfqa)

daq.setVector(f'/{UHFQA}/qas/0/integration/weights/0/real', ch1_w_real)
daq.setVector(f'/{UHFQA}/qas/0/integration/weights/0/imag', ch1_w_imag)

daq.setVector(f'/{UHFQA}/qas/0/integration/weights/1/real', ch2_w_real)
daq.setVector(f'/{UHFQA}/qas/0/integration/weights/1/imag', ch2_w_imag)

### 6) Compile & upload AWG programs and start the program for four repetitions  <a class="anchor" id="chapter6"></a>

In [137]:
UHF_qubitSim_prog = '''
const ampl = 0.5;
 
wave I_q0_state_0 = zeros(128);
wave Q_q0_state_0 = zeros(128);
wave I_q0_state_1 = sine(128, ampl, 0, 2);
wave Q_q0_state_1 = cosine(128, ampl, 0, 2);
 
wave I_q1_state_0 = zeros(128);
wave Q_q1_state_0 = zeros(128);
wave I_q1_state_1 = sine(128, ampl, 0, 4);
wave Q_q1_state_1 = cosine(128, ampl, 0, 4);
 
wave I_state_00 = I_q1_state_0 + I_q0_state_0;
wave Q_state_00 = Q_q1_state_0 + Q_q0_state_0;
 
wave I_state_01 = I_q1_state_0 + I_q0_state_1;
wave Q_state_01 = Q_q1_state_0 + Q_q0_state_1;
 
wave I_state_10 = I_q1_state_1 + I_q0_state_0;
wave Q_state_10 = Q_q1_state_1 + Q_q0_state_0;
 
wave I_state_11 = I_q1_state_1 + I_q0_state_1;
wave Q_state_11 = Q_q1_state_1 + Q_q0_state_1;

var qubitState = getUserReg(0); 

repeat(1024){
//wait(1024);
waitDIOTrigger();
setID(0);



switch (qubitState) {
  case 0: playWave(I_state_00, Q_state_00);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 1: playWave(I_state_01, Q_state_01);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 2: playWave(I_state_10, Q_state_10);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 3: playWave(I_state_11, Q_state_11);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
}
}
'''

HD_rstpulse_prog = '''
setUserReg(0,0);
setUserReg(1,0);
setUserReg(2,0);
setUserReg(3,0);

repeat(512){
// At start of program
waitDIOTrigger();
// Wait for qubit measurement result
waitDIOTrigger();
var res = getDIOTriggered();

}

'''

# Channels to test
channels = [0,1]

# Setup UHF AWG module
awg_uhf = daq.awgModule()
awg_uhf.set('device', UHFQA)
awg_uhf.set('index', 0)
awg_uhf.execute()
awg_uhf.set('awg/enable', 0)

# Setup HD AWG module
awg_hd = daq.awgModule()
awg_hd.set('device', HDAWG)
awg_hd.set('index', 0)
awg_hd.execute()
awg_hd.set('awg/enable', 0)

# Upload HD program
awg_compile_upload_elf(awg_uhf, 0, UHF_qubitSim_prog)

# Upload UHF program
awg_compile_upload_elf(awg_hd, 0, HD_rstpulse_prog)

# PQSC holdoff time in seconds (must be multiple of 10)
daq.setDouble(f'/{PQSC}/execution/repetitions', 4)
daq.setDouble(f'/{PQSC}/execution/holdoff', 100e-3)

### 7) Start sample delay iteration <a class="anchor" id="chapter7"></a>
#### This will take a while due to:
##### The state 11 is being iterated 4 times (each with different sample delays
##### Each PQSC triger hold-off time is 100ms
##### This can take a minimum of 4*100ms = 0.4 seconds for each iteration
##### An additional 0.6 seconds will be added for poll time = 1 second due to any holdoff between resetting and polling 

In [ ]:
# Channels to test
channels = [0,1]

for ch in channels:
    daq.setDouble('/{:s}/qas/0/thresholds/{:d}/level'.format(UHFQA, ch), 0)

# 3 --> both qubits are in excited state
daq.setInt(f'/{UHFQA}/AWGS/0/USERREGS/0', 3)

# Set number of length the QA result monitor should acquire each iteration
daq.setDouble(f'/{UHFQA}/qas/0/result/length', 1)
daq.setDouble(f'/{UHFQA}/qas/0/result/averages', 4)

maxCh1 = 0
maxCh2 = 0
optimumDelay = 0

# Subscribe to result waves
paths = []
for ch in channels:
    path = '/{:s}/qas/0/result/data/{:d}/wave'.format(UHFQA, ch)
    paths.append(path)
daq.subscribe(paths)

# Arm the device
daq.asyncSetInt('/{:s}/awgs/0/single'.format(UHFQA), 1)
daq.syncSetInt('/{:s}/awgs/0/enable'.format(UHFQA), 1)

# Iterate through each sample 
for i in range(0, 1024, 4):
    start_time = time.time()
    # Now we're ready for readout. Enable result unit and start acquisition.
    print(f'Iteration about to start with a sample delay of {i} added')
    daq.setDouble(f'/{UHFQA}/qas/0/delay', i)

    daq.setInt('/{:s}/qas/0/result/reset'.format(UHFQA), 1)
    daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 1)
    daq.sync()
    # Let UHF-AWG relax 
    ##time.sleep(2)

    # PQSC begins sending triggers for UHFQA qubit simulations
    daq.setInt(f'/{PQSC}/execution/enable', 1)

    # Perform acquisition
    # 32 iterations with PQSC hold-off time at 100ms --> 100ms*32 = 3.2seconds
    # An additional code hold-off time of 1 second will also be added to 4.2 seconds

    print('Acquiring data...')
    data = daq.poll(1, 1000, 4, True)
    print('Done.')

    # Stop result unit
    
    daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 0)

    # Now find the absolute value in both integration channels 0 and 1
    absCh1 = np.abs(data[paths[0]][0]['vector'])
    absCh2 = np.abs(data[paths[1]][0]['vector'])

    print(f'Sample delay of {i} produced absolute channel 1 result of {absCh1} and absolute channel 2 result of {absCh2}')

    if(absCh1 > maxCh1 and absCh2 > maxCh2):
        maxCh1 = absCh1
        maxCh2 = absCh2
        optimumDelay = i
        
    print("--- %s seconds ---" % (time.time() - start_time))
daq.unsubscribe(paths)

print(f'Optimum delay found at a sample delay of {optimumDelay}')

In [ ]:
for ch in channels:
    daq.setDouble('/{:s}/qas/0/thresholds/{:d}/level'.format(UHFQA, ch), 0)

# 3 --> both qubits are in excited state
daq.setInt(f'/{UHFQA}/AWGS/0/USERREGS/0', 3)

# Set number of length the QA result monitor should acquire each iteration
daq.setDouble(f'/{UHFQA}/qas/0/result/length', 256)
daq.setDouble(f'/{UHFQA}/qas/0/result/averages', 4)

maxCh1 = 0
maxCh2 = 0
optimumDelay = 0

for i in range(0, 1024, 128):
    
    # Now we're ready for readout. Enable result unit and start acquisition.
    print(f'Iteration about to start with a sample delay of {i} added')
    daq.setDouble(f'/{UHFQA}/qas/0/delay', i)
    
    daq.setInt('/{:s}/qas/0/result/reset'.format(UHFQA), 1)
    daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 1)
    daq.sync()
    #
    # Subscribe to result waves
    paths = []
    for ch in channels:
        path = '/{:s}/qas/0/result/data/{:d}/wave'.format(UHFQA, ch)
        paths.append(path)
    daq.subscribe(paths)
    #
    # Arm the device
    daq.asyncSetInt('/{:s}/awgs/0/single'.format(UHFQA), 1)
    daq.syncSetInt('/{:s}/awgs/0/enable'.format(UHFQA), 1)
    
    
    # Let UHF-AWG relax 
    time.sleep(2)
    
    # PQSC begins sending triggers for UHFQA qubit simulations
    daq.asyncSetInt(f'/{PQSC}/execution/enable', 1)
    
    
    # Perform acquisition
    # 32 iterations with PQSC hold-off time at 100ms --> 100ms*4 = 0.4s
    # An additional code hold-off time of 1 second will also be added to total 1s
    
    #print('Acquiring data...')
    #data = daq.poll(1, 2000, 4, True)
    #print('Done.')

    # Stop result unit
    daq.unsubscribe(paths)
    daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 0)
    
    # Find average of both integration channels 0 and 1 (in complex format)
    avgCh1 = np.average(data[paths[0]][0]['vector'])
    avgCh2 = np.average(data[paths[1]][0]['vector'])
    
    # Now find the absolute value in both integration channels 0 and 1
    absCh1 = np.abs(avgCh1)
    absCh2 = np.abs(avgCh2)
    
    print(f'Sample delay of {i} produced absolute channel 1 result of {absCh1} and absolute chanel 2 result of {absCh2}')
    
    if(absCh1 > maxCh1 and absCh2 > maxCh2):
        maxCh1 = absCh1
        maxCh2 = absCh2
        optimumDelay = i

print(f'Optimum delay found at a sample delay of {optimumDelay}')

In [153]:
#def js_test_QA_sampleDelay_calibration(daq, UHFQA, HDAWG, PQSC):
# Channels to test
channels = [0,1]

# Set QA result monitor length to acquire integrated I/Q values
daq.setInt(f'/{UHFQA}/qas/0/result/source', 7)

# 3 --> both qubits are in excited state
daq.setInt(f'/{UHFQA}/AWGS/0/USERREGS/0', 3)


# Set averaging mode to Sequential
daq.setInt(f'/{UHFQA}/qas/0/result/mode', 1)


# QA should record 9 data points in total
# Each data point averages 4 times : so in total, 9*4 = 36 acquisitions
daq.setDouble(f'/{UHFQA}/qas/0/result/length', 9)
daq.setDouble(f'/{UHFQA}/qas/0/result/averages', 4)
daq.setInt(f'/{UHFQA}/qas/0/delay', 0)

# Reset QA result
daq.setInt('/{:s}/qas/0/result/reset'.format(UHFQA), 1)
daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 1)

# I want to configure QA delay parameter in each of the 9 data point; so 4 PQSC triggers; enabled 9 times
daq.setInt(f'/{PQSC}/execution/repetitions', 4)
daq.sync()

# Subscribe to result waves
paths = []
for ch in channels:
    path = '/{:s}/qas/0/result/data/{:d}/wave'.format(UHFQA, ch)
    paths.append(path)
daq.subscribe(paths)

# Arm the device - UHFQA waits for 1 DIO trigger before startQAResultMonitor() 
daq.asyncSetInt('/{:s}/awgs/0/single'.format(UHFQA), 1)
daq.syncSetInt('/{:s}/awgs/0/enable'.format(UHFQA), 1)
# Arm the HD device - This is not really needed but good to show integrity of QCCS
daq.asyncSetInt('/{:s}/awgs/0/single'.format(HDAWG), 1)
daq.syncSetInt('/{:s}/awgs/0/enable'.format(HDAWG), 1)

time.sleep(2)

# Iterate for 9 result points, 4 averages for every point = 36 total acquisitions
# Between each iteration, change a QA parameter
QAResultAcquired = 0
daq.sync()
for index, i in enumerate(range(0, 1024+128, 128)):
    print(f'Iteration {index}: QA sample delay set to {i}, running 4 PQSC triggers now')
    # Set sample delay value
    daq.syncSetInt(f'/{UHFQA}/qas/0/delay', i)
    time.sleep(1)
    # Acquires 4 data sambles for the delay set above (PQSC repetition set to 4, so 4 result points should be added)
    daq.syncSetInt(f'/{PQSC}/execution/enable', 1)
    timeTaken = 0
    
    while(daq.getInt(f'/{PQSC}/execution/progress') != 1):
        time.sleep(1)
        timeTaken = time + 1
        assert timeTaken < 5, print(f'Took too long')
        
    acquired = daq.getInt(f'/{UHFQA}/qas/0/result/acquired')
    print(f'Iteration {index}: PQSC execution progress finished at 100% - QA Result monitor has acquired {acquired} so far')
        
    
    
    
    
    QAResultAcquired = daq.getInt(f'/{UHFQA}/qas/0/result/acquired')



# Stop result unit
daq.unsubscribe(paths)
daq.setInt('/{:s}/qas/0/result/enable'.format(UHFQA), 0)

ch1results = daq.get(f'/{UHFQA}/QAS/0/RESULT/DATA/0/WAVE', flat = True)
ch2results = daq.get(f'/{UHFQA}/QAS/0/RESULT/DATA/1/WAVE', flat = True)


sampleDelay1 = np.argmax(ch1results)
sampleDelay2 = np.argmax(ch2results)


print(f'Sample delay of {sampleDelay1} and {sampleDelay2}')

Iteration 0: QA sample delay set to 0, running 4 PQSC triggers now
Iteration 0: PQSC execution progress finished at 100% - QA Result monitor has acquired 0 so far
Iteration 1: QA sample delay set to 128, running 4 PQSC triggers now
Iteration 1: PQSC execution progress finished at 100% - QA Result monitor has acquired 4 so far
Iteration 2: QA sample delay set to 256, running 4 PQSC triggers now
Iteration 2: PQSC execution progress finished at 100% - QA Result monitor has acquired 8 so far
Iteration 3: QA sample delay set to 384, running 4 PQSC triggers now
Iteration 3: PQSC execution progress finished at 100% - QA Result monitor has acquired 12 so far
Iteration 4: QA sample delay set to 512, running 4 PQSC triggers now
Iteration 4: PQSC execution progress finished at 100% - QA Result monitor has acquired 16 so far
Iteration 5: QA sample delay set to 640, running 4 PQSC triggers now
Iteration 5: PQSC execution progress finished at 100% - QA Result monitor has acquired 20 so far
Iteration